# Дашборд Кейса №2

## Импорты библиотек

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr  # new
import os  # new

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)

sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

## Вспомогательные функции

Подгрузка csv файла

In [ ]:
def load_data(file):
    """
    Загружает CSV-файл в DataFrame и возвращает данные, их фрагмент и информационные сообщения.

    Функция предназначена для использования в интерактивных приложениях (например, Gradio),
    где пользователь загружает файл через интерфейс. Проверяет наличие файла, читает его
    с автоматическим парсингом дат, и возвращает структурированные данные и текстовые
    сообщения об успешной загрузке и объёме данных.

    Параметры
    ----------
    file : UploadedFile или None
        Объект загруженного файла. Должен иметь атрибут `name`, указывающий путь к файлу.
        Если значение `None`, функция возвращает сообщение об ошибке.

    Возвращает
    -------
    tuple
        Кортеж из четырёх элементов:
        - pd.DataFrame или None:
            Полный DataFrame с данными из CSV. Если файл не загружен — `None`.
        - pd.DataFrame:
            Первые 10 строк данных (`df.head(10)`) для предварительного просмотра.
            При отсутствии файла возвращается пустой DataFrame.
        - str:
            Сообщение о статусе загрузки: успех или ошибка.
        - str или None:
            Дополнительное описание содержимого файла: количество строк и столбцов.
            Если файл не загружен — возвращается `None`.

    Обработка данных
    ----------------
    - Чтение выполняется через `pd.read_csv()` по пути `file.name`.
    - Столбцы `survey_creation_dt` и `survey_response_dt` автоматически преобразуются в тип `datetime`.
    - Имя файла извлекается с помощью `os.path.basename()` для чистоты отображения.

    Примеры использования
    ---------------------
    >>> data, preview, status_msg, desc_msg = load_data(uploaded_file)
    >>> if data is not None:
    ...     st.success(status_msg)
    ...     st.write(desc_msg)
    ...     st.dataframe(preview)
    ... else:
    ...     st.error(status_msg)

    Замечания
    ---------
    - Если указанные столбцы с датами отсутствуют в CSV, pandas выдаст предупреждение.
    - Рекомендуется обернуть `pd.read_csv()` в блок try-except в продвинутых сценариях
      для обработки повреждённых или некорректных файлов.
    - Поддерживает работу только с CSV-файлами. Другие форматы не обрабатываются.
    """
    if file is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате csv", None
    df = pd.read_csv(
        file.name, parse_dates=["survey_creation_dt", "survey_response_dt"]
    )
    filename = os.path.basename(file.name)
    text_status = "Файл " + str(filename) + " успешно загружен"
    text_description = (
        "Файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description

Очистка данных

In [ ]:
def clear_data(df):
    if df is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате CSV.", None

    # 1. Вычисление времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    # 2. Исправление перепутанных дат (где ответ раньше создания)
    negative_before = (df["response_time_days"] < 0).sum()
    text_description = f"Записей с отрицательным временем ответа: {negative_before}.\n"

    df["dates_swapped"] = df["response_time_days"] < 0
    swap = df["dates_swapped"]
    df.loc[swap, ["survey_creation_dt", "survey_response_dt"]] = df.loc[
        swap, ["survey_response_dt", "survey_creation_dt"]
    ].values

    # Пересчёт времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    negative_after = (df["response_time_days"] < 0).sum()
    text_description = text_description + (f"После исправления: {negative_after}. \n")

    # 3. Заполнение пропусков в language
    language_missing = df["language"].isnull().sum()
    df["language"] = df["language"].fillna("RU")
    text_description = text_description + (
        f"Заполнено пропусков в language: {language_missing}. \n"
    )

    text_status = "Файл успешно обработан"
    text_description = text_description + (
        "Актуальный файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description


def how_many_nulls(df):
    mapping = {"survey_creation"}

## Структура дашборда

In [44]:
# структура Дашборда
with gr.Blocks(title="Кейс 2 - Команда 8") as case2:
    gr.Markdown("""
    <div style="text-align:center">

    # Дашборд команды №8  
    Кейс №2. Пенсионный фонд, предоставляющий клиентам цифровые сервисы.

    </div>
    """)
    # df_state для того, чтобы сохранить данные для кнопок
    df_state = gr.State(None)

    # структура дашборда
    with gr.Tabs():
        # Таб с загрузкой данных
        with gr.Tab("Загрузка данных"):
            with gr.Row():
                result_box = gr.Textbox(label="Статус", lines = 3, max_lines= 15, scale = 1)
                description_box = gr.Textbox(label="Описание", lines = 3, max_lines = 15, scale = 3)
            file_input = gr.File(label="Выберите CSV-файл", file_types=[".csv"], scale = 1, height = 100)
            with gr.Row():
                load_button = gr.Button("Загрузить файл csv")
                clear_button = gr.Button("Очистить данные")
            head_df = gr.DataFrame(label="Пример данных", type="pandas", interactive=False, 
                                   column_widths=[120, 160, 160, 70, 70, 70, 70, 90, 110, 110, 110],
                                   wrap = True
                                   )
        # Таб с авторами
        with gr.Tab("Авторы"):
            # Автор 1
            with gr.Row():
                gr.Image(value="team/team_01.png", height=250, width=50)
                gr.Image(value="team/team_02.png", height=250, width=50)
                gr.Image(value="team/team_03.png", height=250, width=50)
            with gr.Row():
                gr.Markdown(
                        """
        **Имя Фамилия 1**  
        Роль: Роль №1

        [LinkedIn](https://linkedin.com/in/author1)  
        [GitHub](https://github.com/author1)
                        """
                    )
                gr.Markdown(
                        """
        **Имя Фамилия 2**  
        Роль: Роль №2

        [Telegram](https://t.me/author2)  
        [GitHub](https://github.com/author2)
                        """
                    )
                gr.Markdown(
                        """
        **Имя Фамилия 3**  
        Роль: Роль №3

        [VK](https://vk.com/author3)  
        [GitHub](https://github.com/author3)
                        """
                    )

                

    # операции

    # Загрузка CSV
    load_button.click(
        load_data,
        inputs=[file_input],
        outputs=[df_state, head_df, result_box, description_box],
    )
    # Очистка данных
    clear_button.click(
        clear_data,
        inputs=[df_state],
        outputs=[df_state, head_df, result_box, description_box],
    )



In [45]:
case2.launch(share=False, inline=True)

* Running on local URL:  http://127.0.0.1:7910
* To create a public link, set `share=True` in `launch()`.
